In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load the IMDB dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

# Pad sequences to ensure all reviews are the same length
X_train = pad_sequences(X_train, maxlen=200)
X_test = pad_sequences(X_test, maxlen=200)

# Convert to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 20s 1us/step


In [3]:
from tensorflow.keras import models, layers

def build_model():
    model = models.Sequential()
    model.add(layers.Embedding(input_dim=10000, output_dim=128))  # Removed input_length
    model.add(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model()


In [4]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 58s 89ms/step - accuracy: 0.6986 - loss: 0.5553 - val_accuracy: 0.8208 - val_loss: 0.4048
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 56s 90ms/step - accuracy: 0.8169 - loss: 0.4212 - val_accuracy: 0.7786 - val_loss: 0.4646
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 57s 91ms/step - accuracy: 0.8341 - loss: 0.3829 - val_accuracy: 0.8126 - val_loss: 0.4384
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 55s 88ms/step - accuracy: 0.8884 - loss: 0.2783 - val_accuracy: 0.8410 - val_loss: 0.3899
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 54s 87ms/step - accuracy: 0.9143 - loss: 0.2218 - val_accuracy: 0.8572 - val_loss: 0.3847
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 54s 86ms/step - accuracy: 0.9303 - loss: 0.1822 - val_accuracy: 0.8552 - val_loss: 0.3978
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 53s 85ms/step - accuracy: 0.9476 - loss: 0.1436 - val_accuracy: 0.8460 - val_loss: 0.4186
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3650s 6s/step - accuracy: 0.9573 - loss: 0.1224 - 

In [5]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.8491 - loss: 0.5080
Accuracy: 84.97200012207031


In [6]:
import json

word_index = imdb.get_word_index()

def decode_review(encoded_review):
    reverse_word_index = {value: key for (key, value) in word_index.items()}
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])
    return decoded_review

# Decode the first 3 reviews in the test set
sample_reviews = [decode_review(X_test[i]) for i in range(3)]

# Preprocess reviews
def preprocess_reviews(reviews):
    sequences = imdb.get_word_index()
    indices = [[sequences.get(word, 0) for word in review.split()] for review in reviews]
    return pad_sequences(indices, maxlen=200)

preprocessed_reviews = preprocess_reviews(sample_reviews)


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [7]:
predictions = model.predict(preprocessed_reviews)

# Output the predictions
for review, prediction in zip(sample_reviews, predictions):
    sentiment = 'Positive' if prediction > 0.5 else 'Negative'
    print(f'Review: "{review}"\nPredicted Sentiment: {sentiment}\n')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Review: "? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? please give this one a miss br br ? ? and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite ? so all you madison fans give this a miss"
Predicted Sentiment: Positive

Review: "psychological ? it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of her darker chara